In [1]:
!pip install dlt
!dlt --version
!pip install pyarrow==11.0.0
!pip install "dlt[duckdb]"
!pip install streamlit
!pip install parquet
!pip install gcloud
!pip install dlt[gs]
!pip install google-cloud-bigquery-storage

dlt 0.4.4


zsh:1: no matches found: dlt[gs]


In [1]:
import dlt
import duckdb

import json
import requests
import parquet
import pandas as pd
import numpy as np

import os
import glob
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
        
import urllib3, socket
from urllib3.connection import HTTPConnection

## API AND GCS CREDENTIALS

In [13]:
#API authorization and ifno
api_token_wc = "***"
my_headers = {"Authorization" : f"Bearer {api_token_wc}"}
api_url = "https://staging.data-api.bepro11.com"

HTTPConnection.default_socket_options = (HTTPConnection.default_socket_options + [(socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000), #1MB in byte 
                                        (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000)])

# GCS Credentials: Only need this if you're running this code locally.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../keys/credentials.json'
os.environ['GOOGLE_CLOUD_PROJECT'] = '***'

In [3]:
def get_match_event_data_paggineted(match_id):
    
    limit = 100
    page_number = 1

    
    while True:   
        offset = (page_number-1)*limit
        # API request
        response = requests.get(f"{api_url}/api/matches/{match_id}/event_data?limit={limit}&offset={offset}", headers=my_headers)
        
        # Raise an HTTPError for bad responses
        response.raise_for_status()  
        
        # Get response
        page_json = response.json()
        
        if page_json:
            if (page_json["result"] != []):
                yield page_json["result"]
                page_number += 1
            else:
                break
            
        else:
            # No more data, break the loop
            break


## LOAD FROM API TO BIG QUERY

### GET LEAGUE AND SEASONS OF THE TOURNAMENT

In [3]:
#API request
response = requests.get(f"{api_url}/api/leagues", headers=my_headers)
        
# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()

#print(page_json["result"][0])

league_id = page_json["result"][0]["id"]
season_ids = page_json["result"][0]["season_ids"]
season_id = season_ids[0]

print("League ID", league_id)
print("Season ID", season_id)

League ID 1534
Season ID 2172


### GET ALL EVENTS FROM ALL MATCHES OF A LEAGUE

In [5]:
#API request
response = requests.get(f"{api_url}/api/matches?season={season_id}", headers=my_headers)
        
# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()
# Get matches
matches = page_json["result"]
#print(matches)

#define the pipeline
pipeline_match_events = dlt.pipeline(
    pipeline_name='match_events_pipeline',
    destination='bigquery',
    dataset_name='match_events_dataset',
    import_schema_path="schemas/import",
    export_schema_path="schemas/export"
)

#for each match, sotre info in table
for match_info in matches:
    match_id = match_info["id"]
    print(match_id)
    load_match_events_info = pipeline_match_events.run(get_match_event_data_paggineted(match_id), write_disposition="append", table_name="match_events_table")

81384


2024-03-30 22:47:08,014|[WARNING              ]|51717|8593865408|dlt|pipeline.py|run:610|The pipeline `run` method will now load the pending load packages. The data you passed to the run function will not be loaded. In order to do that you must run the pipeline again


81383
81287
81286
80856
80868
80855
80854
80684
80683
80682
80681
80680
80679
80678
80677
57218
57217
57224
57223
79023
57206
57212
57211
57200
57199
79022
57203
79021
57194
57191
57190
57222
57215
57221
57216
57205
57210
57209
79020
57197
57202
57198
79018
57193
57188
57189
79017
57213
57219
57220
57214
57207
79016
57204
57208
79015
57196
57201
57195
79014
57187
57192
57186


### GET INFO OF TEAMS IN THE TOURNAMENT 

In [5]:
#API request
response = requests.get(f"{api_url}/api/teams?season={season_id}", headers=my_headers)
        
# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()
# Get teams
teams_list = page_json["result"]
#display(teams_list)

#define the pipeline
pipeline_teams = dlt.pipeline(
    pipeline_name='teams_pipeline',
    destination='bigquery',
    dataset_name='match_events_dataset',
    import_schema_path="schemas/import",
    export_schema_path="schemas/export"
)

#for each match, sotre info in table
#load_teams_info = pipeline_teams.run(teams_list, write_disposition="append", table_name="teams_table")

### GET INFO OF PLAYERS OF ALL THE TEAMS IN THE TOURNAMENT 

In [12]:
#define the pipeline
pipeline_players = dlt.pipeline(
    pipeline_name='players_pipeline',
    destination='bigquery',
    dataset_name='match_events_dataset',
    import_schema_path="schemas/import",
    export_schema_path="schemas/export"
)


for team in teams_list:
    #API request
    team_id = team["id"]
    response = requests.get(f"{api_url}/api/players?team={team_id}", headers=my_headers)
        
    # Raise an HTTPError for bad responses
    response.raise_for_status()  
        
    # Get response
    page_json = response.json()
    # Get teams
    players_list = page_json["result"]
    #display(players_list)
    
    load_players_info = pipeline_players.run(players_list, write_disposition="append", table_name="players_table")

PipelineStepFailed: Pipeline execution failed at stage sync with exception:

<class 'dlt.common.configuration.exceptions.ConfigFieldMissingException'>
Following fields are missing: ['project_id', 'private_key', 'client_email'] in configuration with spec GcpServiceAccountCredentials
	for field "project_id" config providers and keys were tried in following order:
		In Environment Variables key PLAYERS_PIPELINE__DESTINATION__BIGQUERY__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key PLAYERS_PIPELINE__DESTINATION__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key PLAYERS_PIPELINE__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DESTINATION__BIGQUERY__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DESTINATION__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key CREDENTIALS__PROJECT_ID was not found.
	for field "private_key" config providers and keys were tried in following order:
		In Env

### GET INFO OF ALL THE MATCHES IN THE TOURNAMENT 

In [8]:
#API request
response = requests.get(f"{api_url}/api/matches?season={season_id}", headers=my_headers)
        
# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()
# Get matches
matches = page_json["result"]
#print(matches)

#define the pipeline
pipeline_match_info = dlt.pipeline(
    pipeline_name='match_info_pipeline',
    destination='bigquery',
    dataset_name='match_events_dataset',
    import_schema_path="schemas/import",
    export_schema_path="schemas/export"
)

load_match_info = pipeline_match_info.run(matches, write_disposition="append", table_name="match_info_table")

### GET TEAMS STATS FOR ALL THE MATCHES IN THE TOURNAMENT

In [14]:
#API request
response = requests.get(f"{api_url}/api/matches?season={season_id}", headers=my_headers)

# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()

# Get matches
matches = page_json["result"]

#define the team stats pipeline
pipeline_team_stats_match = dlt.pipeline(
    pipeline_name='pipeline_match_team_stats',
    destination='bigquery',
    dataset_name='match_team_stats_dataset',
    import_schema_path="schemas/import",
    export_schema_path="schemas/export"
)


#for each match, sotre info in table
for match_info in matches:
    match_id = match_info["id"]
    print("-------------------------match_id----------------------------")
    print(match_id)
    print("-------------------------------------------------------------")

    response_team_stats = requests.get(f"{api_url}/api/matches/{match_id}/team_stats", headers=my_headers)

    # Raise an HTTPError for bad responses
    response_team_stats.raise_for_status()  

    # Get response
    response_team_stats_json = response_team_stats.json()
    response_team_stats_json['result'][0]['match_id'] = match_id
    response_team_stats_json['result'][1]['match_id'] = match_id

    #print("-------------------------team stats response result----------------------------")
    #print(response_team_stats_json['result'])
    #print("-------------------------------------------------------------")
    #print("-------------------------team stats result[match_id]----------------------------")
    #print(response_team_stats_json['result'][0])
    #print(response_team_stats_json['result'][1])
    #print("-------------------------------------------------------------")
    
    load_match_team_stats_info = pipeline_team_stats_match.run(response_team_stats_json["result"], write_disposition="append", table_name="match_team_stats_table")

-------------------------match_id----------------------------
81384
-------------------------------------------------------------
-------------------------match_id----------------------------
81383
-------------------------------------------------------------
-------------------------match_id----------------------------
81287
-------------------------------------------------------------
-------------------------match_id----------------------------
81286
-------------------------------------------------------------
-------------------------match_id----------------------------
80856
-------------------------------------------------------------
-------------------------match_id----------------------------
80868
-------------------------------------------------------------
-------------------------match_id----------------------------
80855
-------------------------------------------------------------
-------------------------match_id----------------------------
80854
----------------------

### GET PLAYERS STATS FOR ALL THE MATCHES IN THE TOURNAMENT

In [20]:
#API request
response = requests.get(f"{api_url}/api/matches?season={season_id}", headers=my_headers)

# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()
# Get matches
matches = page_json["result"]

#define the players stats pipeline
pipeline_match_players_stats = dlt.pipeline(
    pipeline_name='pipeline_match_players_stats',
    destination='bigquery',
    dataset_name='match_players_stats_dataset',
    import_schema_path="schemas/import",
    export_schema_path="schemas/export"
)

#for each match, sotre info in table
for match_info in matches:
    match_id = match_info["id"]
    print("-------------------------match_id----------------------------")
    print(match_id)
    print("-------------------------------------------------------------")
    response_players_stats = requests.get(f"{api_url}/api/matches/{match_id}/player_stats", headers=my_headers)

    # Raise an HTTPError for bad responses
    response_players_stats.raise_for_status()  

    # Get response
    response_players_stats_json = response_players_stats.json()
    for player in response_players_stats_json['result']:
        player["match_id"] = match_id
        #print("-------------------------player----------------------------")
        #print(player)
        #print("------------------------------------------------------------")
    #print("-------------------------players stats response result----------------------------")
    #print(response_players_stats_json['result'])
    print("-------------------------------------------------------------")
    
    load_match_players_stats_info = pipeline_match_players_stats.run(response_players_stats_json["result"], write_disposition="append", table_name="match_players_stats_table")

-------------------------match_id----------------------------
81384
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
81383
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
81287
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
81286
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
80856
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id-------

-------------------------------------------------------------
-------------------------match_id----------------------------
79018
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
57193
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
57188
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
57189
-------------------------------------------------------------
-------------------------------------------------------------
-------------------------match_id----------------------------
79017
-------------------------------------------------------------
----------------------------------------

## GET SEQUENCE OF PLAYS FOR ALL THE TEAMS PER MATCH

In [14]:
#API request
response = requests.get(f"{api_url}/api/matches?season={season_id}", headers=my_headers)

# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()
# Get matches
matches = page_json["result"]

#define the players stats pipeline
pipeline_match_sequence = dlt.pipeline(
    pipeline_name='pipeline_match_sequence',
    destination='bigquery',
    dataset_name='match_sequence_dataset',
    import_schema_path="schemas/import",
    export_schema_path="schemas/export"
)

#Initializing sequence_id
sequence_id = 0

#for each match, sotre info in table
for match_info in matches:
    match_id = match_info["id"]
    print("-------------------------match_id----------------------------")
    print(match_id)
    print("-------------------------------------------------------------")
    response_match_sequence = requests.get(f"{api_url}/api/matches/{match_id}/sequence_data", headers=my_headers)

    # Raise an HTTPError for bad responses
    response_match_sequence.raise_for_status()  

    # Get response
    response_match_sequence_json = response_match_sequence.json()
    for sequence in response_match_sequence_json['result']:
        sequence["match_id"] = match_id
        sequence["sequence_id"] = sequence_id
        sequence_id = sequence_id + 1
        print("-------------------------sequence----------------------------")
        print(sequence)
        print("------------------------------------------------------------")
    print("-------------------------match sequence response result----------------------------")
    print(response_match_sequence_json['result'])
    print("-------------------------------------------------------------")
    
    load_match_sequence_info = pipeline_match_sequence.run(response_match_sequence_json['result'], write_disposition="append", table_name="match_sequence_table")

-------------------------match_id----------------------------
81384
-------------------------------------------------------------
-------------------------sequence----------------------------
{'start_time': 2311, 'end_time': 7048, 'team_id': 3338, 'event_period': 'FIRST_HALF', 'event_ids': [81614078, 81614079, 81614134, 81614135], 'match_id': 81384, 'sequence_id': 0}
------------------------------------------------------------
-------------------------sequence----------------------------
{'start_time': 7048, 'end_time': 7048, 'team_id': None, 'event_period': 'FIRST_HALF', 'event_ids': [81614135], 'match_id': 81384, 'sequence_id': 1}
------------------------------------------------------------
-------------------------sequence----------------------------
{'start_time': 17895, 'end_time': 19712, 'team_id': 14618, 'event_period': 'FIRST_HALF', 'event_ids': [81615893, 81615894, 81615223], 'match_id': 81384, 'sequence_id': 2}
------------------------------------------------------------
----

PipelineStepFailed: Pipeline execution failed at stage sync with exception:

<class 'dlt.common.configuration.exceptions.ConfigFieldMissingException'>
Following fields are missing: ['project_id', 'private_key', 'client_email'] in configuration with spec GcpServiceAccountCredentials
	for field "project_id" config providers and keys were tried in following order:
		In Environment Variables key PIPELINE_MATCH_SEQUENCE__DESTINATION__BIGQUERY__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key PIPELINE_MATCH_SEQUENCE__DESTINATION__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key PIPELINE_MATCH_SEQUENCE__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DESTINATION__BIGQUERY__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DESTINATION__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key CREDENTIALS__PROJECT_ID was not found.
	for field "private_key" config providers and keys were tried in foll

## FOR TESTING

In [7]:
def get_match_event_data_paggineted_test(match_id):
    
    limit = 100
    page_number = 1

    
    while page_number < 100:   
        offset = (page_number-1)*limit
        # API request
        response = requests.get(f"{api_url}/api/matches/{match_id}/event_data?limit={limit}&offset={offset}", headers=my_headers)
        
        # Raise an HTTPError for bad responses
        response.raise_for_status()  
        
        # Get response
        page_json = response.json()
        
        if page_json:
            if (page_json["result"] != []):
                yield page_json["result"]
                page_number += 1
            else:
                break
            
        else:
            # No more data, break the loop
            break

In [4]:
#API request
response = requests.get(f"{api_url}/api/leagues", headers=my_headers)
        
# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()

#print(page_json["result"][0])

league_id = page_json["result"][0]["id"]
season_ids = page_json["result"][0]["season_ids"]
season_id = season_ids[0]

print("League ID", league_id)
print("Season ID", season_id)

League ID 1534
Season ID 2172


In [11]:
#Match id for testing
match_id = 81384

#define the pipeline
duckdb_pipeline_match_info = dlt.pipeline(pipeline_name='duckdb_match_info_pipeline',destination='duckdb',dataset_name='duckdb_match_events_dataset')

info = duckdb_pipeline_match_info.run(get_match_event_data_paggineted_test(match_id),table_name="duckdb_match_info_table", write_disposition="append")

print(info)

Pipeline duckdb_match_info_pipeline load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset duckdb_match_events_dataset
The duckdb destination used duckdb:////Users/matiasroubaud/Desktop/de_zoomcamp/data-engineering-zoomcamp-main/final_project/duckdb_match_info_pipeline.duckdb location to store data
Load package 1709995004.37103 is LOADED and contains no failed jobs


In [12]:
conn = duckdb.connect(f"{duckdb_pipeline_match_info.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{duckdb_pipeline_match_info.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌──────────────────────────────────────┐
│                 name                 │
│               varchar                │
├──────────────────────────────────────┤
│ _dlt_loads                           │
│ _dlt_pipeline_state                  │
│ _dlt_version                         │
│ duckdb_match_info_table              │
│ duckdb_match_info_table__event_types │
└──────────────────────────────────────┘

In [14]:
# and the data of each table

print("\n\n\n match_event table below:")

match_events = conn.sql("SELECT * FROM duckdb_match_info_table").df()
display(match_events)

print("\n\n\n match_event_types table below:")

match_event_info = conn.sql("SELECT * FROM duckdb_match_info_table__event_types").df()
display(match_event_info)




 match_event table below:


,id,event_period,event_time,match_id,team_id,player_id,x,y,relative_event__id,relative_event__event_time,relative_event__player_id,relative_event__x,relative_event__y,_dlt_load_id,_dlt_id,ball_position__x,ball_position__y,xg
0,81614078,FIRST_HALF,2311,81384,3338.0,41509.0,0.493644,0.498626,81614079.0,3285.0,311353.0,0.508475,0.379121,1709995004.37103,puRkZltae+DNcA,NaN,NaN,NaN
1,81614079,FIRST_HALF,3285,81384,3338.0,311353.0,0.508475,0.379121,NaN,NaN,NaN,NaN,NaN,1709995004.37103,b0+9Rj7uRZwAGQ,NaN,NaN,NaN
2,81614134,FIRST_HALF,4478,81384,3338.0,311353.0,0.578390,0.412088,81614135.0,7048.0,41500.0,0.118644,0.736264,1709995004.37103,41FOlUi3yxcSuQ,NaN,NaN,NaN
3,81614135,FIRST_HALF,7048,81384,3338.0,41500.0,0.118644,0.736264,NaN,NaN,NaN,NaN,NaN,1709995004.37103,kVaP7AkiDA6liw,NaN,NaN,NaN
4,81615893,FIRST_HALF,17895,81384,14618.0,328427.0,1.000000,0.193318,81615894.0,19153.0,328451.0,0.982193,0.278370,1709995004.37103,WnhuajBQv9t9Kw,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2796,81691106,EXTRA_SECOND_HALF,7435249,81384,3338.0,41500.0,0.167373,0.909341,NaN,NaN,NaN,NaN,NaN,1709995004.37103,Je+za0R5K9ltSg,NaN,NaN,NaN
2797,81655596,EXTRA_SECOND_HALF,7436073,81384,14618.0,328430.0,0.826783,0.060545,NaN,NaN,NaN,NaN,NaN,1709995004.37103,43kkl6/AsXBq3w,NaN,NaN,NaN
2798,81691122,EXTRA_SECOND_HALF,7436326,81384,3338.0,41500.0,0.177966,0.954670,NaN,NaN,NaN,NaN,NaN,1709995004.37103,mFBWPY8lo7QhTQ,NaN,NaN,NaN
2799,81655617,EXTRA_SECOND_HALF,7438349,81384,14618.0,328444.0,0.651869,0.077297,NaN,NaN,NaN,NaN,NaN,1709995004.37103,u2RVViuo0VR3uw,NaN,NaN,NaN





 match_event_types table below:


,event_type,outcome,cross,key_pass,assist,_dlt_parent_id,_dlt_list_idx,_dlt_id,sub_event_type,body_part
0,Pass,Successful,False,False,False,puRkZltae+DNcA,0,h2bgHJOUO1aFMQ,None,None
1,Pass Received,None,NaN,NaN,NaN,b0+9Rj7uRZwAGQ,0,u35E7igC1WHaXQ,None,None
2,Pass,Successful,False,False,False,41FOlUi3yxcSuQ,0,cshPFSP5cij5oA,None,None
3,Pass Received,None,NaN,NaN,NaN,kVaP7AkiDA6liw,0,I6nX5jJsRA8vHQ,None,None
4,Error,None,NaN,NaN,NaN,kVaP7AkiDA6liw,1,ctH3WTIvfytCPw,None,None
...,...,...,...,...,...,...,...,...,...,...
3175,Take-On,Successful,NaN,NaN,NaN,Je+za0R5K9ltSg,0,/TYkc7TWTdtlyw,None,None
3176,Tackle,Unsuccessful,NaN,NaN,NaN,43kkl6/AsXBq3w,0,PU/KDtOP2qdjKQ,None,None
3177,Take-On,Unsuccessful,NaN,NaN,NaN,mFBWPY8lo7QhTQ,0,oCrxQkXxdLhpcw,None,None
3178,Intervention,None,NaN,NaN,NaN,u2RVViuo0VR3uw,0,Gg0bN39It19U0g,None,None


In [22]:
# API authorization and ifno
my_token = "***"
my_headers = {"Authorization" : f"Bearer {my_token}"}
api_url = "https://staging.data-api.bepro11.com"

# API request
response = requests.get(f"{api_url}/api/matches?season={season_id}", headers=my_headers)

# Raise an HTTPError for bad responses
response.raise_for_status()  
        
# Get response
page_json = response.json()

# Print result
page_json

{'count': 64,
 'next': None,
 'previous': None,
 'result': [{'id': 81384,
   'start_time': '2022-12-18 16:00:00+01:00',
   'round': {'id': 23794, 'name': '4'},
   'season': {'id': 2172,
    'name': '2022',
    'season_group_name': '2022',
    'league_id': 1534},
   'home_team': {'id': 14618,
    'name': 'Argentina',
    'name_en': 'Argentina',
    'iso_country_code': 'AR'},
   'away_team': {'id': 3338,
    'name': 'France',
    'name_en': 'France',
    'iso_country_code': 'FR'},
   'full_time': 90,
   'half_time_duration': 45,
   'extra_full_time': 30,
   'detail_match_result': {'home_team_score': 3, 'away_team_score': 3},
   'extra_match_result': {'id': 205,
    'home_team_score': 4,
    'away_team_score': 2,
    'type': 'PSO'},
   'venue': None,
   'is_analysis_finished': True,
   'live_analysing': False,
   'analysis_completed_datetime': '2022-12-19T15:15:33.638381+00:00'},
  {'id': 81383,
   'start_time': '2022-12-17 16:00:00+01:00',
   'round': {'id': 23794, 'name': '4'},
   'seas